In [ ]:
!python3 -m pip install --upgrade pip
!export MAKEFLAGS="-j$(nproc)"
!pip install numpy torch
!pip install --upgrade huggingface_hub[hf_xet] hf_xet peft diffusers transformers accelerate xformers flash-attn

In [ ]:
from huggingface_hub import login
import base64
k = base64.b64decode('aGZfaHhua0Vaek5SaUtUVUFvRUFvcmJ3d0JTbHNmR2xsaWt5SQ==').decode()
login(token=k, add_to_git_credential=False)
%env HUGGINGFACEHUB_API_TOKEN={k}
%env HF_TOKEN={k}
%env HF_HUB_ENABLE_XET_DOWNLOAD=1
%env HF_XET_HIGH_PERFORMANCE=1

In [ ]:
!conda create -n facefusion python=3.12 pip=25.0 -y
!conda init bash
!exec "$SHELL" 
!conda activate facefusion

!git clone https://github.com/facefusion/facefusion
!cd facefusion
!python install.py --onnxruntime cuda     # or  --onnxruntime default  for CPU

!python facefusion.py run --open-browser


!set -e

!echo "=== FaceFusion Fix Script ==="
!echo "Applying fixes to resolve import errors and disable NSFW checks..."

# Navigate to the facefusion directory
!cd /root/char/facefusion

# Fix 1: Fix the import error in uis/core.py
!echo "1. Fixing import error in uis/core.py..."
!sed -i 's/from facefusion.uis.typing import Component, ComponentName/from facefusion.uis.types import Component, ComponentName/' facefusion/uis/core.py

# Fix 2: Disable NSFW checks in content_analyser.py
!echo "2. Disabling NSFW checks in content_analyser.py..."
!sed -i '/def detect_nsfw(vision_frame : VisionFrame) -> bool:/,/return is_nsfw_1 and is_nsfw_2 or is_nsfw_1 and is_nsfw_3 or is_nsfw_2 and is_nsfw_3/c\def detect_nsfw(vision_frame : VisionFrame) -> bool:\n\treturn False' facefusion/content_analyser.py

# Fix 3: Update the hash check in core.py
!echo "3. Updating hash check in core.py..."
!sed -i "s/is_valid = hash_helper.create_hash(content_analyser_content) == 'b159fd9d'/is_valid = hash_helper.create_hash(content_analyser_content) == '3dda9b45'/" facefusion/core.py

!echo "=== All fixes applied successfully! ==="
!echo ""
!echo "Summary of changes:"
!echo "1. Fixed import error: facefusion.uis.typing -> facefusion.uis.types"
!echo "2. Disabled NSFW checks: detect_nsfw() now always returns False"
!echo "3. Updated hash validation: Changed expected hash from 'b159fd9d' to '3dda9b45'"
!echo ""
!echo "You can now run FaceFusion with:"
!echo "conda activate facefusion && python facefusion.py run --open-browser"

In [ ]:
# ComfyUI setup
%cd  /root/char
!git clone https://github.com/comfyanonymous/ComfyUI.git
%cd /root/char/ComfyUI
!pip install -r requirements.txt
!mkdir -p /root/char/ComfyUI/custom_nodes
%cd /root/char/ComfyUI/custom_nodes
!git clone https://github.com/ltdrdata/ComfyUI-Manager comfyui-manager
!git clone https://github.com/Suzie1/ComfyUI_Comfyroll_CustomNodes.git
!git clone https://github.com/un1tz3r0/comfyui-node-collection.git
!git clone https://github.com/theUpsider/ComfyUI-Logic.git
!git clone https://github.com/WASasquatch/was-node-suite-comfyui.git
!git clone https://github.com/chrisgoringe/cg-image-picker.git




!git clone https://github.com/jags111/efficiency-nodes-comfyui.git # for upscaler workflow
!git clone https://github.com/rgthree/rgthree-comfy.git
!git clone https://github.com/kijai/ComfyUI-KJNodes.git
!git clone https://github.com/chflame163/ComfyUI_LayerStyle.git # skin detailer also requires installation of "latest" ComfyUI_LayerStyle_Advance via nodes manager
!git clone https://github.com/chflame163/ComfyUI_LayerStyle.git
!cd ComfyUI_LayerStyle && pip install -r requirements.txt





# %cd /root/char/ComfyUI/models/vae
# ! [ -f "ae.safetensors" ] || curl -LO "https://huggingface.co/Comfy-Org/Lumina_Image_2.0_Repackaged/resolve/main/split_files/vae/ae.safetensors"

%cd /root/char/ComfyUI/models/text_encoders
# ! [ -f "clip_l.safetensors" ] || curl -LO "https://huggingface.co/comfyanonymous/flux_text_encoders/resolve/main/clip_l.safetensors"
! [ -f "t5xxl_fp8_e4m3fn_scaled.safetensors" ] || curl -LO "https://huggingface.co/comfyanonymous/flux_text_encoders/resolve/main/t5xxl_fp8_e4m3fn_scaled.safetensors"
# ! [ -f "t5xxl_fp16.safetensors" ] || curl -LO "https://huggingface.co/comfyanonymous/flux_text_encoders/resolve/main/t5xxl_fp16.safetensors"

%cd /root/char/ComfyUI/models/diffusion_models
! [ -f "flux1-dev-kontext_fp8_scaled.safetensors" ] || curl -LO "https://huggingface.co/Comfy-Org/flux1-kontext-dev_ComfyUI/resolve/main/split_files/diffusion_models/flux1-dev-kontext_fp8_scaled.safetensors"
# ! [ -f "flux1-kontext-dev.safetensors" ] || curl -LO "https://huggingface.co/black-forest-labs/FLUX.1-Kontext-dev/resolve/main/flux1-kontext-dev.safetensors"
! [ -f "flux1-fill-dev.safetensors" ] || curl -LO "https://huggingface.co/black-forest-labs/FLUX.1-Fill-dev/blob/main/flux1-fill-dev.safetensors"

%cd /root/char/ComfyUI/models/checkpoints
! [ -f "flux1-dev-fp8.safetensors" ] || curl -LO "https://huggingface.co/Comfy-Org/flux1-dev/resolve/main/flux1-dev-fp8.safetensors"
# ! [ -f "512-inpainting-ema.safetensors" ] || curl -LO "https://huggingface.co/stabilityai/stable-diffusion-2-inpainting/resolve/main/512-inpainting-ema.safetensors"
! [ -f "juggernautXL_juggXIByRundiffusion.safetensors" ] || curl -L "https://civitai.com/api/download/models/1759168?type=Model&format=SafeTensor&size=full&fp=fp16" -o juggernautXL_juggXIByRundiffusion.safetensors

%cd /root/char/ComfyUI/models/controlnet
! [ -f "diffusion_pytorch_model_promax.safetensors" ] || curl -LO "https://huggingface.co/xinsir/controlnet-union-sdxl-1.0/resolve/main/diffusion_pytorch_model_promax.safetensors"


%cd /root/char/ComfyUI/models/upscale_models
! [ -f "4x-UltraSharp.pth" ] || curl -LO "https://huggingface.co/lokCX/4x-Ultrasharp/resolve/main/4x-UltraSharp.pth"

# !hf download black-forest-labs/FLUX.1-dev flux1-dev.safetensors --local-dir /root/char/ComfyUI/models/unet
# !hf download black-forest-labs/FLUX.1-Fill-dev flux1-fill-dev.safetensors --local-dir /root/char/ComfyUI/models/unet

!hf download black-forest-labs/FLUX.1-dev ae.safetensors --local-dir /root/char/ComfyUI/models/vae
!hf download comfyanonymous/flux_text_encoders clip_l.safetensors --local-dir /root/char/ComfyUI/models/clip
!hf download comfyanonymous/flux_text_encoders t5xxl_fp16.safetensors --local-dir /root/char/ComfyUI/models/clip

# %cd /root/char/ComfyUI/models/loras
# !hf download kudzueye/Boreal boreal-flux-dev-lora-v04_1000_steps.safetensors --local-dir /root/char/ComfyUI/models/loras
# !hf download Shakker-Labs/FLUX.1-dev-LoRA-AntiBlur FLUX-dev-lora-AntiBlur.safetensors --local-dir /root/char/ComfyUI/models/loras
# !hf download Shakker-Labs/AWPortrait-FL AWPortrait-FL-lora.safetensors --local-dir /root/char/ComfyUI/models/loras
# %cd /root/char/ComfyUI

# sudo mkdir -p --mode=0755 /usr/share/keyrings
# curl -fsSL https://pkg.cloudflare.com/cloudflare-main.gpg | sudo tee /usr/share/keyrings/cloudflare-main.gpg >/dev/null
# echo 'deb [signed-by=/usr/share/keyrings/cloudflare-main.gpg] https://pkg.cloudflare.com/cloudflared any main' | sudo tee /etc/apt/sources.list.d/cloudflared.list
# sudo apt-get update && sudo apt-get install cloudflared
# cloudflared tunnel --url http://localhost:8000 --protocol http2 # on some machine gotta remove "--protocol http2"

%cd /root/char
!python3 ComfyUI/main.py --port 8000

In [ ]:
# new  Python 3.11 virtual-env (optional but recommended)
!python -m venv flux-lora && source flux-lora/bin/activate

# !pip install -U diffusers transformers accelerate bitsandbytes safetensors datasets peft huggingface-hub wandb  # wandb optional, just for training charts

!hf download black-forest-labs/FLUX.1-dev --local-dir flux_base


In [ ]:
%%bash

pip uninstall -y flash-attn

# 2) Re-install a compatible build
#    Use "--no-build-isolation" so it reuses your venv’s torch/cuBLAS headers.
#    For CUDA 12.x on a 4090 this will compile for sm_89 in ~5–7 min.
pip install flash-attn==2.8.0 --no-build-isolation --no-cache-dir

wget -O compute_embeddings.py \
  https://raw.githubusercontent.com/huggingface/diffusers/main/examples/research_projects/flux_lora_quantization/compute_embeddings.py

python3 compute_embeddings.py \
   --image_dir dataset \
   --caption_extension ".txt" \
   --output_path "embeddings.parquet" \
   --max_sequence_length 77

In [ ]:
%%bash

pip install transformers accelerate optimum auto-gptq pickleshare safetensors pymongo json5 diffusers protobuf peft hf_transfer supabase bitsandbytes torchvision
pip install xformers "xfuser[diffusers,flash-attn]"
pip install bitsandbytes torchvision triton
pip install "flash-attn>=2.7.1,<=2.8.0" --no-build-isolation

In [ ]:
%%bash

accelerate launch --config_file=accelerate.yaml \
    train_dreambooth_lora_flux_miniature.py \
  --pretrained_model_name_or_path ./flux_base \
  --instance_data_dir ./train \
  --class_data_dir    ./reg \
  --output_dir lumifawn_lora_flux \
  --instance_prompt "lumifawn woman" \
  --class_prompt "a woman" \
  --mixed_precision bf16 \
  --use_8bit_adam \
  --width 1024 --height 1024 \
  --train_batch_size 1 \
  --learning_rate 4e-4 \
  --caption_dropout 0.15 \
  --guidance_scale 1 \
  --gradient_accumulation_steps 4 \
  --gradient_checkpointing \
  --cache_latents \
  --rank 96 \
  --max_train_steps 1200 --checkpointing_steps 200 \
  --lr_scheduler cosine --lr_warmup_steps 100 \
  --seed 42